-------
-------
-------
-------
-------
-------
# Wide Residual, derinliği şişirmeden,geniş ve güçlü dönüşümleri residual ile eğitilebilir hale getirerek,bazı görevlerde çok derin ağlardan daha verimli sonuçlar verebilir.
-------
-------
-------
-------
-------
-------

# Wide Residual Block (Wide ResNet) — Temelden İleri Seviyeye Notlar

> Bu notlar, **Wide Residual Block** (WRB) ve **Wide ResNet (WRN)** fikrini, klasik residual bağlantıdan başlayıp mimari kararlar, motivasyon ve pratik PyTorch uygulamasına kadar adım adım anlatır.  
> Amaç: Konuyu **her baktığında hatırlayabileceğin** netlikte, **karmaşıklaştırmadan** ama **eksik bırakmadan** özetlemek.


## 1) Önce temel: Residual bağlantı (değişmeyen iskelet)

Residual blokların çekirdeği:

\[
y = x + F(x)
\]

- **Skip/Shortcut path**: \(x\) (identity) veya boyutlar uymuyorsa projection.
- **Main path**: Öğrenilebilir dönüşüm \(F(x)\) (Conv/BN/ReLU vb.).

Bu iskelet, **Basic**, **Pre-Act**, **Bottleneck**, **ResNeXt**, **Wide** gibi tüm varyantlarda aynı kalır.  
Farklar, yalnızca \(F(x)\)'in **nasıl parametrikleştirildiği** ve **kanal/stride** düzenindedir.


## 2) Wide Residual Block nedir?

**Wide Residual Block**, residual fikrini ( \(y=x+F(x)\) ) bozmaz.  
Yaptığı şey:

> **Derinliği aşırı artırmak yerine kanalları (width) artırarak kapasiteyi büyütmek.**

“Wide”, uzamsal genişlik değil; **kanal sayısı** demektir:
- 64 kanal → 128/256 kanal gibi **genişletme**
- Daha az katmanla daha yüksek temsil gücü hedefi

Bu yaklaşımın yaygın formu **Wide ResNet (WRN)**’tir.


## 3) Neden ortaya çıktı? (motivasyon)

Klasik (çok derin) ResNet’lerde pratik gözlemler:
1. Aşırı derinlikte bazı bloklar öğrenme sırasında **identity'e yakın** davranabilir.
2. Derinlik arttıkça eğitim/çalıştırma maliyeti büyür; GPU verimliliği her zaman lineer artmaz.
3. “Daha derin = daha iyi” yaklaşımı her zaman optimum değildir.

Wide ResNet fikri:
- Derinliği ölçülü tut (ör. 16–40 bandı),
- **Kanal genişliğini artır** (widen factor ile),
- Daha güçlü bloklarla daha iyi doğruluk / eğitim verimliliği hedefle.


## 4) Wide ResNet isimlendirmesi: WRN-\(d\)-\(k\)

- **\(d\)**: derinlik parametresi (blok/layer sayısı düzeni)
- **\(k\)**: widen factor (genişlik çarpanı)

Örnek: **WRN-28-10**
- \(d=28\)
- \(k=10\): kanal sayıları temel düzenin 10 katı ölçeğinde

Pratik yorum:
- \(k\) büyüdükçe kanallar artar → kapasite artar → FLOPs/parametre artar.
- \(d\) büyüdükçe blok sayısı artar → maliyet artar.


## 5) Wide Residual Block’un tipik iç yapısı

Yaygın blok formu **pre-activation** düzeniyle yazılır:

```bash
x
│
├─ BN → ReLU → Conv 3×3  (geniş kanallar)
│
├─ BN → ReLU → Conv 3×3  (geniş kanallar)
│
└─ Skip: identity veya 1×1 projection
        ↓
      Toplama (x + F(x))
```

### Neden pre-activation?
BN/ReLU’nun toplamadan önce gelmesi, gradyan akışını pratikte iyileştirir (eğitim stabilitesi).

### Dropout nerede?
Bazı WRN varyantlarında 1. ve 2. conv arasına **dropout** eklenir (overfitting azaltmak için).


## 6) “Wide” tam olarak nerede? (kanal genişliği)

Wide yaklaşımında ana kontrol düğmesi:

\[
\text{out\_ch} = k \times \text{base\_ch}
\]

- **base_ch**: stage’in temel kanal sayısı
- **k (widen factor)**: kanalı kaç kat büyüteceğin

Örnek:
- base_ch = 64, k = 2 → out_ch = 128
- base_ch = 64, k = 4 → out_ch = 256


## 7) Wide Residual Block vs diğer residual varyantlar (kısa kıyas)

| Varyant | Temel fikir | \(F(x)\) içi | Kapasiteyi artırma yolu |
|---|---|---|---|
| Basic | Basit residual | 3×3 → 3×3 | Derinlik / stage kanalı |
| Pre-Act | Optimizasyon iyileştirme | (BN+ReLU) + conv | Eğitim stabilitesi |
| Bottleneck | Hesap verimliliği | 1×1 → 3×3 → 1×1 | Derinliği ucuza artırma |
| ResNeXt | Cardinality | 1×1 → **grouped** 3×3 → 1×1 | **groups** ile çoklu temsil |
| **Wide** | Width | (genelde) 3×3 → 3×3 (geniş) | **kanalları büyütme (k)** |

**Özet:** Wide = “tek dönüşümü güçlendir”  
ResNeXt = “çoklu küçük dönüşüm (cardinality)”


## 8) Hangi alanlarda kullanılır?

### 8.1) Sınıflandırma (Classification)
- Wide ResNet özellikle CIFAR tarzı benchmark’larda güçlü sonuçlarıyla bilinir.
- Overfitting riskine karşı dropout/augmentasyon ile iyi çalışır.

### 8.2) Detection (Object Detection)
- Backbone olarak kullanılabilir (feature extractor).
- Ancak kanal genişliği arttıkça FLOPs ve latency artar.
- Detection’da residual yapılar genelde backbone’da yoğunlaşır; head tarafında aşırı residual çoğu zaman gereksizdir.

### 8.3) Segmentation
- Güçlü backbone ihtiyacında kullanılabilir; maliyet dengesi önemlidir.

**Pratik kural:** Accuracy odaklıysanız wide avantajlı olabilir; gerçek zamanlı/edge odakta daha ölçülü olunmalıdır.


## 9) PyTorch ile Wide Residual Block (WRB) — Referans Uygulama

Aşağıdaki kod:
- Pre-activation düzeni kullanır (BN→ReLU→Conv)
- Gerekirse skip için projection (1×1) ekler
- İsteğe bağlı dropout içerir


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WideResidualBlock(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, stride: int = 1, dropout_p: float = 0.0):
        super().__init__()

        self.bn1 = nn.BatchNorm2d(in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)

        self.dropout = nn.Dropout(p=dropout_p) if dropout_p and dropout_p > 0 else None

        self.bn2 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        
        # Bu kanal değişimini burada yapıyor:
                # base → wide (64 → 128)
        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False)

    def forward(self, x):
        # Pre-activation
        out = self.bn1(x)
        out = F.relu(out, inplace=True)

        # Skip path (identity veya projection)
        skip = x if self.proj is None else self.proj(x)

        # Main path
        out = self.conv1(out)

        #
        if self.dropout is not None:
            out = self.dropout(out)

        out = self.bn2(out)
        out = F.relu(out, inplace=True)
        out = self.conv2(out)

        # Residual toplama
        out = out + skip
        return out


### 9.1) Hızlı shape testi
- `stride=1` → uzamsal boyut değişmez  
- `stride=2` → uzamsal boyut yarıya düşer (skip projection devreye girer)


In [2]:
x = torch.randn(2, 64, 56, 56)

blk_same = WideResidualBlock(in_ch=64, out_ch=64, stride=1, dropout_p=0.0)
y_same = blk_same(x)

blk_down = WideResidualBlock(in_ch=64, out_ch=128, stride=2, dropout_p=0.0)
y_down = blk_down(x)

print("x     :", x.shape)
print("y_same:", y_same.shape)
print("y_down:", y_down.shape)


x     : torch.Size([2, 64, 56, 56])
y_same: torch.Size([2, 64, 56, 56])
y_down: torch.Size([2, 128, 28, 28])


## 10) Wide fikrini widen factor ile stage tasarımına bağlamak

Basit bir stage örneği:
- base kanal: 64
- widen factor: \(k=2\) → stage kanal: 128

Genişlik stratejisi: aynı derinlikte daha fazla kanal kullanarak temsil gücünü artırır.

> Not: Genişlik artışı parametre/FLOPs’ı hızlı büyütebilir. Detection için dengeli seçilir.


## 11) Wide Residual Block'u modele bağlamak (mini backbone örneği)

Aşağıdaki mini model:
- Stem ile 3→64 kanal çıkarır
- Ardından 2 adet WideResidualBlock uygular
- Sonunda global pooling ile çıktı üretir (classifier iskeleti)

Detection tarafında aynı mantıkla bloklar backbone’a yerleştirilir; üstüne neck + head eklenir.


In [ ]:
class TinyWRNBackbone(nn.Module):
    def __init__(self, widen_factor: int = 2, dropout_p: float = 0.0):
        super().__init__()
        base = 64
        wide = base * widen_factor

        self.stem = nn.Sequential(
            nn.Conv2d(3, base, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(base),
            nn.ReLU(inplace=True),
        )
        # 3 kanal = renk

        self.block1 = WideResidualBlock(in_ch=base, out_ch=wide, stride=1, dropout_p=dropout_p)
        self.block2 = WideResidualBlock(in_ch=wide, out_ch=wide, stride=1, dropout_p=dropout_p)

        self.pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.stem(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool(x)
        return x

m = TinyWRNBackbone(widen_factor=2, dropout_p=0.0)
inp = torch.randn(2, 3, 64, 64)
out = m(inp)
print("out:", out.shape)


out: torch.Size([2, 128, 1, 1])


## 12) Detection (YOLO) tarafında “nerede kullanılır?” (pratik rehber)

YOLO gibi detector’larda mimari üç parçaya ayrılır:
1. **Backbone**: feature extractor (residual blokların en doğal yeri)
2. **Neck**: multi-scale feature fusion (FPN/PAN; hafif residual bazen olur)
3. **Head**: sınıflandırma + bbox regresyonu (genelde sade olmalı)

Wide Residual için öneri:
- Backbone’un orta/derin stage’lerinde, ölçülü widen factor ile kullanmak mantıklı.
- Çok erken stage’de (yüksek çözünürlükte) “aşırı wide” latency’yi hızlı artırır.
- Head içinde çok derin residual çoğu zaman gereksizdir.


## 13) Sık yapılan hatalar (kontrol listesi)

- **Skip ve F(x) shape uyumsuzluğu**: `in_ch != out_ch` veya `stride!=1` ise projection şart.
- **Aşırı genişlik**: Detection’da FPS düşürür; k’yı küçük tutarak başlamak daha güvenlidir.
- **Normalization/activation yerleşimi**: Pre-act düzeni eğitim stabilitesi açısından pratikte güçlüdür.


## 14) Mini özet

- Residual iskelet değişmez: \(y=x+F(x)\)
- Wide yaklaşımı: **derinlik yerine kanal genişliğiyle kapasite artırma**
- En sık kullanım: sınıflandırma backbone’u
- Detection’da kullanım: backbone içinde ölçülü (özellikle orta/derin katmanlar)
